The US Geological Survey publishes a list of Strategic Minerals ( https://www.usgs.gov/news/national-news-release/us-geological-survey-releases-2022-list-critical-minerals ). Having a secure supply of these minerals is essential to our security and economic prosperity. However many of these minerals are sourced from outside of the US. This assignment is to develop a reference catalog of the source or sources of each of these minerals and a judgement on the reliability of each source under stressed circumstance (e.g. war, economic crisis, etc.)

Notes:

You will need to identify a source or sources for each of the minerals in the 2022 List of Critical Minerals

You will need to categorize each source country as an ally, a competitor or a neutral party.

You will need to develop data visualizations that tell the story of source dependency and shortfall impact.

This assignment is due at the end of week fourteen of the semester

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import comtradeapicall as ct
import requests
import json
import re

## List of Minerals

In [84]:
minerals = {
    "Aluminium": "used in almost all sectors of the economy",
    "Antimony": "used in lead-acid batteries and flame retardants",
    "Arsenic": "used in semi-conductors",
    "Barite": "used in hydrocarbon production",
    "Beryllium": "used as an alloying agent in aerospace and defense industries",
    "Bismuth": "used in medical and atomic research",
    "Cerium": "used in catalytic converters, ceramics, glass, metallurgy, and polishing compounds",
    "Cesium": "used in research and development",
    "Chromium": "used primarily in stainless steel and other alloys",
    "Cobalt": "used in rechargeable batteries and superalloys",
    "Dysprosium": "used in permanent magnets, data storage devices, and lasers",
    "Erbium": "used in fiber optics, optical amplifiers, lasers, and glass colorants",
    "Europium": "used in phosphors and nuclear control rods",
    "Fluorspar": "used in the manufacture of aluminum, cement, steel, gasoline, and fluorine chemicals",
    "Gadolinium": "used in medical imaging, permanent magnets, and steelmaking",
    "Gallium": "used for integrated circuits and optical devices like LEDs",
    "Germanium": "used for fiber optics and night vision applications",
    "Graphite": "used for lubricants, batteries, and fuel cells",
    "Hafnium": "used for nuclear control rods, alloys, and high-temperature ceramics",
    "Holmium": "used in permanent magnets, nuclear control rods, and lasers",
    "Indium": "used in liquid crystal display screens",
    "Iridium": "used as coating of anodes for electrochemical processes and as a chemical catalyst",
    "Lanthanum": "used to produce catalysts, ceramics, glass, polishing compounds, metallurgy, and batteries",
    "Lithium": "used for rechargeable batteries",
    "Lutetium": "used in scintillators for medical imaging, electronics, and some cancer therapies",
    "Magnesium": "used as an alloy and for reducing metals",
    "Manganese": "used in steelmaking and batteries",
    "Neodymium": "used in permanent magnets, rubber catalysts, and in medical and industrial lasers",
    "Nickel": "used to make stainless steel, superalloys, and rechargeable batteries",
    "Niobium": "used mostly in steel and superalloys",
    "Palladium": "used in catalytic converters and as a catalyst agent",
    "Platinum": "used in catalytic converters",
    "Praseodymium": "used in permanent magnets, batteries, aerospace alloys, ceramics, and colorants",
    "Rhodium": "used in catalytic converters, electrical components, and as a catalyst",
    "Rubidium": "used for research and development in electronics",
    "Ruthenium": "used as catalysts, as well as electrical contacts and chip resistors in computers",
    "Samarium": "used in permanent magnets, as an absorber in nuclear reactors, and in cancer treatments",
    "Scandium": "used for alloys, ceramics, and fuel cells",
    "Tantalum": "used in electronic components, mostly capacitors and in superalloys",
    "Tellurium": "used in solar cells, thermoelectric devices, and as alloying additive",
    "Terbium": "used in permanent magnets, fiber optics, lasers, and solid-state devices",
    "Thulium": "used in various metal alloys and in lasers",
    "Tin": "used as protective coatings and alloys for steel",
    "Titanium": "used as a white pigment or metal alloys",
    "Tungsten": "primarily used to make wear-resistant metals",
    "Vanadium": "primarily used as alloying agent for iron and steel",
    "Ytterbium": "used for catalysts, scintillometers, lasers, and metallurgy",
    "Yttrium": "used for ceramic, catalysts, lasers, metallurgy, and phosphors",
    "Zinc": "primarily used in metallurgy to produce galvanized steel",
    "Zirconium": "used in the high-temperature ceramics and corrosion-resistant alloys"
}

## Data

https://www.usgs.gov/centers/national-minerals-information-center/minerals-yearbook-metals-and-minerals

https://comtradeplus.un.org/  
https://pypi.org/project/comtradeapicall/

In [85]:
key = pd.read_json('data/api_keys.json', typ='series')['uncomtrade']

In [146]:
df = ct.getFinalData(
    key, typeCode='C', freqCode='A', clCode='HS', period='2023', 
    reporterCode=None, cmdCode='2606', flowCode='X', partnerCode=0, 
    partner2Code=0, customsCode=None, motCode=None
)

In [154]:
df.loc[df.reporterCode == 792, :].T

,82,86,88,90,91,94,96,97,98,99
typeCode,C,C,C,C,C,C,C,C,C,C
freqCode,A,A,A,A,A,A,A,A,A,A
refPeriodId,20230101,20230101,20230101,20230101,20230101,20230101,20230101,20230101,20230101,20230101
refYear,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023
refMonth,52,52,52,52,52,52,52,52,52,52
period,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023
reporterCode,792,792,792,792,792,792,792,792,792,792
reporterISO,None,None,None,None,None,None,None,None,None,None
reporterDesc,None,None,None,None,None,None,None,None,None,None
flowCode,X,X,X,X,X,X,X,X,X,X


In [138]:
df

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate
0,C,A,20230101,2023,52,2023,36,None,None,X,...,3.747951e+10,False,4.108179e+10,False,None,1.115857e+09,1.115857e+09,0,False,True
1,C,A,20230101,2023,52,2023,56,None,None,X,...,1.398584e+04,False,0.000000e+00,False,None,1.243027e+04,1.243027e+04,0,False,True
2,C,A,20230101,2023,52,2023,70,None,None,X,...,4.238141e+07,False,0.000000e+00,False,None,2.567743e+06,2.567743e+06,0,False,True
3,C,A,20230101,2023,52,2023,72,None,None,X,...,3.323050e+05,False,0.000000e+00,False,None,3.710096e+05,3.710096e+05,0,False,True
4,C,A,20230101,2023,52,2023,76,None,None,X,...,NaN,False,0.000000e+00,False,None,1.755592e+08,1.755592e+08,0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,C,A,20230101,2023,52,2023,834,None,None,X,...,3.000000e+01,False,0.000000e+00,False,None,6.400000e+00,6.400000e+00,0,False,True
96,C,A,20230101,2023,52,2023,792,None,None,X,...,1.250000e+02,False,0.000000e+00,False,None,5.400000e+01,5.400000e+01,0,False,True
97,C,A,20230101,2023,52,2023,792,None,None,X,...,2.625080e+09,False,0.000000e+00,False,None,8.077765e+07,8.077765e+07,0,False,True
98,C,A,20230101,2023,52,2023,792,None,None,X,...,4.278831e+06,False,0.000000e+00,False,None,3.056350e+05,3.056350e+05,0,False,True


In [147]:
df.reporterCode.value_counts()

reporterCode
792    10
328     8
276     6
70      6
76      5
826     5
710     4
442     4
688     4
703     4
288     4
508     4
124     4
300     3
428     3
458     3
752     2
203     2
834     2
72      2
36      2
579     1
616     1
376     1
528     1
440     1
56      1
372     1
246     1
208     1
191     1
156     1
152     1
842     1
Name: count, dtype: int64

In [90]:
ct.listReference()

,category,variable,description,fileuri
0,dataitem,Trade data items,List of data items/variables used in UN Comtrade,https://comtradeapi.un.org/files/v1/app/refere...
1,freq,Frequency,The time interval at which observations occur,https://comtradeapi.un.org/files/v1/app/refere...
2,flow,Trade Flow,"Trade flow or sub-flow (exports, re-exports, i...",https://comtradeapi.un.org/files/v1/app/refere...
3,mot,Mode of Transports,The mode of transport used when goods enter or...,https://comtradeapi.un.org/files/v1/app/refere...
4,partner,Partner country/area,The partner country or geographic area for the...,https://comtradeapi.un.org/files/v1/app/refere...
5,reporter,Reporter,The country or geographic area to which the me...,https://comtradeapi.un.org/files/v1/app/refere...
6,qtyunit,Units of quantity,The unit for quantity and alternate quantity d...,https://comtradeapi.un.org/files/v1/app/refere...
7,mos,Mode of Supply,The mode of supply on delivery of services (tr...,https://comtradeapi.un.org/files/v1/app/refere...
8,cmd:B4,Product,The classification of BEC Rev.4 – goods and se...,https://comtradeapi.un.org/files/v1/app/refere...
9,cmd:B5,Product,The classification of BEC Rev.5 – goods or ser...,https://comtradeapi.un.org/files/v1/app/refere...


In [106]:
ref_df = ct.listReference()
url = ref_df.loc[ref_df.category == 'cmd:HS', 'fileuri'].values[0]
response = requests.get(url)
response_json = json.loads(response.text)

commodities_codes_df = pd.DataFrame(response_json['results'])

minerals_df = pd.DataFrame()

for mineral in minerals:
    # temp_df = commodities_codes_df.loc[commodities_codes_df.text.str.contains(fr'{mineral}.* ore', case=False), :]
    temp_df = commodities_codes_df.loc[commodities_codes_df.text.str.contains(f'{mineral}', case=False), :].copy()
    temp_df['mineral'] = mineral
    if len(temp_df) == 0:
        print(f'{mineral} not found')
    else:
        minerals_df = pd.concat([minerals_df, temp_df])
    
minerals_df

Barite not found
Cesium not found
Dysprosium not found
Erbium not found
Europium not found
Holmium not found
Lanthanum not found
Lutetium not found
Neodymium not found
Praseodymium not found
Rubidium not found
Samarium not found
Terbium not found
Thulium not found
Ytterbium not found


,id,text,parent,isLeaf,aggrLevel,standardUnitAbbr,mineral
1437,2510,2510 - Natural calcium phosphates; natural alu...,25,0,4,n/a,Aluminium
1438,251010,"251010 - Natural calcium phosphates, natural a...",2510,1,6,kg,Aluminium
1439,251020,"251020 - Natural calcium phosphates, natural a...",2510,1,6,kg,Aluminium
1534,2606,2606 - Aluminium ores and concentrates,26,0,4,n/a,Aluminium
1535,260600,260600 - Aluminium ores and concentrates,2606,1,6,kg,Aluminium
...,...,...,...,...,...,...,...
6095,810931,"810931 - Zirconium; waste and scrap, containin...",8109,1,6,kg,Zirconium
6096,810939,"810939 - Zirconium; waste and scrap, containin...",8109,1,6,kg,Zirconium
6097,810990,"810990 - Zirconium; other than unwrought, n.e....",8109,1,6,kg,Zirconium
6098,810991,"810991 - Zirconium; other than unwrought, n.e....",8109,1,6,kg,Zirconium


In [108]:
url = ref_df.loc[ref_df.category == 'reporter', 'fileuri'].values[0]
response = requests.get(url)
response_json = json.loads(response.text)
reporter_df = pd.DataFrame(response_json['results'])
reporter_df

,id,text,reporterCode,reporterDesc,reporterNote,reporterCodeIsoAlpha2,reporterCodeIsoAlpha3,entryEffectiveDate,isGroup,entryExpiredDate
0,4,Afghanistan,4,Afghanistan,Afghanistan,AF,AFG,1900-01-01T00:00:00,False,NaN
1,8,Albania,8,Albania,Albania,AL,ALB,1900-01-01T00:00:00,False,NaN
2,12,Algeria,12,Algeria,Algeria,DZ,DZA,1900-01-01T00:00:00,False,NaN
3,20,Andorra,20,Andorra,Andorra,AD,AND,1900-01-01T00:00:00,False,NaN
4,24,Angola,24,Angola,Angola,AO,AGO,1900-01-01T00:00:00,False,NaN
...,...,...,...,...,...,...,...,...,...,...
250,887,Yemen,887,Yemen,Yemen,YE,YEM,1991-01-01T00:00:00,False,NaN
251,890,Yugoslavia (...1991),890,Yugoslavia (...1991),Former Yugoslavia,YU,YUG,1900-01-01T00:00:00,False,1991-12-31T00:00:00
252,894,Zambia,894,Zambia,Zambia,ZM,ZMB,1965-01-01T00:00:00,False,NaN
253,836,Zanzibar and Pemba Isd (...1964),836,Zanzibar and Pemba Isd (...1964),Former Zanzibar and Pemba Island,NaN,_ZP,1900-01-01T00:00:00,False,1964-12-31T00:00:00


In [129]:
reporter_df.loc[reporter_df.text == 'Afghanistan', 'id'].values[0]

4

In [128]:
url = ref_df.loc[ref_df.category == 'partner', 'fileuri'].values[0]
response = requests.get(url)
response_json = json.loads(response.text)
partner_df = pd.DataFrame(response_json['results'])
partner_df

,id,text,PartnerCode,PartnerDesc,partnerNote,PartnerCodeIsoAlpha2,PartnerCodeIsoAlpha3,entryEffectiveDate,isGroup,entryExpiredDate
0,4,Afghanistan,4,Afghanistan,Afghanistan,AF,AFG,1900-01-01T00:00:00,False,NaN
1,472,"Africa CAMEU region, nes",472,"Africa CAMEU region, nes","Africa CAMEU region, not elsewhere specified",NaN,_AC,1900-01-01T00:00:00,False,NaN
2,248,Åland Islands,248,Åland Islands,NaN,AX,ALA,1900-01-01T00:00:00,False,NaN
3,8,Albania,8,Albania,Albania,AL,ALB,1900-01-01T00:00:00,False,NaN
4,12,Algeria,12,Algeria,Algeria,DZ,DZA,1900-01-01T00:00:00,False,NaN
...,...,...,...,...,...,...,...,...,...,...
305,887,Yemen,887,Yemen,Yemen,YE,YEM,1991-01-01T00:00:00,False,NaN
306,890,Yugoslavia (...1991),890,Yugoslavia (...1991),Former Yugoslavia,YU,YUG,1900-01-01T00:00:00,False,1991-12-31T00:00:00
307,894,Zambia,894,Zambia,Zambia,ZM,ZMB,1965-01-01T00:00:00,False,NaN
308,836,Zanzibar and Pemba Isd (...1964),836,Zanzibar and Pemba Isd (...1964),Former Zanzibar and Pemba Island,NaN,_ZP,1900-01-01T00:00:00,False,1964-12-31T00:00:00


In [136]:
partner_df.loc[290:, :]

,id,text,PartnerCode,PartnerDesc,partnerNote,PartnerCodeIsoAlpha2,PartnerCodeIsoAlpha3,entryEffectiveDate,isGroup,entryExpiredDate
290,858,Uruguay,858,Uruguay,Uruguay,UY,URY,1900-01-01T00:00:00,False,NaN
291,849,US Misc. Pacific Isds,849,US Misc. Pacific Isds,US Miscellaneous Pacific Islands,PU,PUS,1900-01-01T00:00:00,False,NaN
292,850,US Virgin Isds (...1980),850,US Virgin Isds (...1980),US Virgin Islands,VI,VIR,1900-01-01T00:00:00,False,1980-12-31T00:00:00
293,842,USA,842,USA,"USA, Puerto Rico and US Virgin Islands",US,USA,1981-01-01T00:00:00,False,NaN
294,841,USA and Puerto Rico (...1980),841,USA and Puerto Rico (...1980),USA and Puerto Rico,US,USA,1900-01-01T00:00:00,False,1980-12-31T00:00:00
295,810,USSR (...1990),810,USSR (...1990),Former USSR,SU,SUN,1900-01-01T00:00:00,False,1990-12-31T00:00:00
296,860,Uzbekistan,860,Uzbekistan,Uzbekistan,UZ,UZB,1992-01-01T00:00:00,False,NaN
297,548,Vanuatu,548,Vanuatu,Vanuatu,VU,VUT,1900-01-01T00:00:00,False,NaN
298,862,Venezuela,862,Venezuela,Venezuela,VE,VEN,1900-01-01T00:00:00,False,NaN
299,704,Viet Nam,704,Viet Nam,Viet Nam,VN,VNM,1975-01-01T00:00:00,False,NaN


In [150]:
partner_df[partner_df.id == 792]

,id,text,PartnerCode,PartnerDesc,partnerNote,PartnerCodeIsoAlpha2,PartnerCodeIsoAlpha3,entryEffectiveDate,isGroup,entryExpiredDate
279,792,Türkiye,792,Türkiye,Türkiye,TR,TUR,1900-01-01T00:00:00,False,NaN


In [97]:
metadata = ct.getMetadata(key, typeCode='C', freqCode='A', clCode='HS', period='2023', reporterCode=None, showHistory=False)

In [98]:
metadata[metadata.reporterCode == 276]

,datasetCode,typeCode,freqCode,period,reporterCode,reporterDescription,currency,importConvFactor,exportConvFactor,tradeSystem,...,importValuation,exportValuation,importPartnerCountry,exportPartnerCountry,importPartner2Country,exportPartner2Country,publicationNote,publicationDate,publicationDateShort,isLatestPublication
59,20276202301202100,C,A,2023,276,Germany,EUR,1.08126862,1.08126862,Special,...,CIF,FOB,Origin,Last Known Destination,Consignment,N/A,New Data: Data for this period is published fo...,2024-02-21T15:45:24.82,2024-02-21,True


In [99]:
metadata.currency.unique()

array(['EUR', 'XAF', 'SEK', 'GBP', 'USD', 'MYR', 'ZAR', 'AUD', 'ZMK',
       'NGN', 'GYD', 'NAD', 'BBD', 'AWG', 'MOP', 'XCD', 'STD', 'XOF',
       'ISK', 'BZD', 'PAB', 'CAD', 'DKK', 'BGL', 'NZD', 'MGA', 'BWP',
       'NOK', 'BND', 'MUR', 'CHF', 'GHC'], dtype=object)

In [117]:
df['reporter'] = df.reporterCode.apply(lambda x: reporter_df.loc[reporter_df.id == x, 'text'].values[0])

In [124]:
df.loc[0,:]

typeCode                             C
freqCode                             A
refPeriodId                   20230101
refYear                           2023
refMonth                            52
period                            2023
reporterCode                        76
reporterISO                       None
reporterDesc                      None
flowCode                             X
flowDesc                          None
partnerCode                         32
partnerISO                        None
partnerDesc                       None
partner2Code                         0
partner2ISO                       None
partner2Desc                      None
classificationCode                  H6
classificationSearchCode            HS
isOriginalClassification          True
cmdCode                           2606
cmdDesc                           None
aggrLevel                         None
isLeaf                            None
customsCode                        C00
customsDesc              

In [127]:
df.loc[0:20, ['reporter', 'partnerCode']]

,reporter,partnerCode
0,Brazil,32
1,Brazil,68
2,Brazil,124
3,Brazil,156
4,Brazil,0
5,Brazil,208
6,Brazil,792
7,Brazil,699
8,Brazil,276
9,Brazil,156


In [122]:
df.cmdCode.unique()

array(['2606'], dtype=object)

In [120]:
reporter_df.text.T

0                           Afghanistan
1                               Albania
2                               Algeria
3                               Andorra
4                                Angola
                     ...               
250                               Yemen
251                Yugoslavia (...1991)
252                              Zambia
253    Zanzibar and Pemba Isd (...1964)
254                            Zimbabwe
Name: text, Length: 255, dtype: object